In [271]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import warnings
import statsmodels.formula.api as smf
from sklearn import linear_model
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from math import sqrt
warnings.filterwarnings('ignore')

In [272]:
df = pd.read_csv('Environmental_impact_cleaned.csv')
df.head(3)

,ISIN,Year,CompanyName,Country,Industry(Exiobase),EnvironmentalIntensity(Sales),EnvironmentalIntensity(OpInc),TotalEnvironmentalCost,WorkingCapacity,FishProductionCapacity,CropProductionCapacity,MeatProductionCapacity,Biodiversity,AbioticResources,Waterproductioncapacity(Drinkingwater&IrrigationWater),WoodProductionCapacity,SDG1.5,SDG2.1,SDG2.2,SDG2.3,SDG2.4,SDG3.3,SDG3.4,SDG3.9,SDG6,SDG12.2,SDG14.1,SDG14.2,SDG14.3,SDG14.c,SDG15.1,SDG15.2,SDG15.5,%Imputed,Env_intensity,industry_avg,industry_avg_year,Industry_indicator_year,Environmental_Growth
0,DE0005545503,2016,1&1 DRILLISCH AG,Germany,Post and telecommunications (64),-0.07%,-0.82%,"-539,318","-525,027",-169,"-7,009","-1,630",-27,-878,"-4,714",135,"-234,989","-166,914","-166,795","-1,752","-1,752","-27,366","65,960",-142,"-4,714",-878,-5,-1,-77,-6,67,67,-22,23%,-0.0007,-0.020506,-0.020740,1,NaN
1,GB00B1YW4409,2010,3I GROUP PLC,United Kingdom,"Financial intermediation, except insurance and...",-0.12%,-0.11%,"-1,055,812","-1,032,103",-277,"-13,751","-3,221",-47,-562,"-5,953",102,"-463,300","-295,103","-294,949","-3,438","-3,438","-47,957","59,044",-74,"-5,953",-562,-4,0,-133,-4,51,51,-43,10%,-0.0012,-0.028537,-0.006402,1,NaN
2,GB00B1YW4409,2011,3I GROUP PLC,United Kingdom,"Financial intermediation, except insurance and...",-0.16%,-0.16%,"-961,875","-940,402",-246,"-12,525","-2,935",-42,-424,"-5,378",77,"-421,928","-264,714","-264,579","-3,131","-3,131","-42,961","44,515",-56,"-5,378",-424,-3,0,-119,-3,38,38,-39,9%,-0.0016,-0.028537,-0.009838,1,33.333333


In [273]:
df = df.loc[:,['CompanyName','Year','Industry(Exiobase)','Env_intensity','Environmental_Growth']]
df.head()

,CompanyName,Year,Industry(Exiobase),Env_intensity,Environmental_Growth
0,1&1 DRILLISCH AG,2016,Post and telecommunications (64),-0.0007,NaN
1,3I GROUP PLC,2010,"Financial intermediation, except insurance and...",-0.0012,NaN
2,3I GROUP PLC,2011,"Financial intermediation, except insurance and...",-0.0016,33.333333
3,3I GROUP PLC,2012,"Financial intermediation, except insurance and...",-0.0015,-6.250000
4,3M COMPANY,2010,Activities of membership organisation n.e.c. (91),-0.0790,NaN



# Industries Regressions-- Past years Environment Intensity



Firstly, we filter industries more than 3 companies

In [274]:
df_industry = df.groupby('Industry(Exiobase)').count()['CompanyName'].reset_index()
industries = df_industry[df_industry['CompanyName'] > 3]['Industry(Exiobase)']
df_industry_count4 = df[df['Industry(Exiobase)'].isin(industries)]
df_industry_count4.head()

,CompanyName,Year,Industry(Exiobase),Env_intensity,Environmental_Growth
0,1&1 DRILLISCH AG,2016,Post and telecommunications (64),-0.0007,NaN
1,3I GROUP PLC,2010,"Financial intermediation, except insurance and...",-0.0012,NaN
2,3I GROUP PLC,2011,"Financial intermediation, except insurance and...",-0.0016,33.333333
3,3I GROUP PLC,2012,"Financial intermediation, except insurance and...",-0.0015,-6.250000
4,3M COMPANY,2010,Activities of membership organisation n.e.c. (91),-0.0790,NaN


Define function for time series model by industries

In [275]:
df_c = df_industry_count4.copy()
def calculateIndustriesRegressions(outcomeYear, pastYears, df_c):
    industry_regressions = {}
    for i in np.unique(df_c['Industry(Exiobase)']):
        for year in years:
            years.sort()
            data = df_c[(df_c['Industry(Exiobase)'] == i) & (df_c['Year'] == year)]
            data = data.loc[:,['CompanyName','Env_intensity']]
            data.rename(columns={'Env_intensity': f'Env_intensity_{year}'}, inplace=True) 
            if(year == min(years)):
                data1 = pd.DataFrame(data)
            else:
                data2 = pd.merge(data1, data, on=["CompanyName"])
                data1 = data2.copy()
        if (data2.shape[0])>6:
            data3 = df_c[df_c['Year'] == outcomeYear]
            data3 = data3[['CompanyName','Env_intensity']]
            data3.rename(columns={'Env_intensity': f'Env_intensity_{outcomeYear}'}, inplace=True) 
            data3 = pd.merge(data3, data2, on=["CompanyName"])

            filter_col = [col for col in data3 if ((col.startswith('Env_intensity') and not(col.endswith(f'{outcomeYear}'))))]
            outcome_col = [col for col in data3 if (col.startswith('Env_intensity') and col.endswith(f'{outcomeYear}'))]
            X=data3[filter_col]
            y=data3[outcome_col]


            regr = linear_model.LinearRegression()
            X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
            regr.fit(X_train, y_train)
            y_train_pred = regr.predict(X_train)
            y_pred = regr.predict(X_test)
        else: 
            continue
        industry_regressions[i] = {'OutcomeYear':outcomeYear, 'MSE_test': metrics.mean_squared_error(y_test, y_pred),'RMSE_test':sqrt(metrics.mean_squared_error(y_test, y_pred)),'Intercept': regr.intercept_,'Coefficient': regr.coef_, 
                                   'R2_score':metrics.r2_score(y_test, y_pred)}
    data_items = industry_regressions.items()
    data_list = list(data_items)

    df = pd.DataFrame(data_list)
    df=pd.concat([df.drop(columns=df.columns[1], axis=1), df.iloc[:,1].apply(pd.Series)], axis=1)
    return df

## Input data from 2016 to 2018 for independent variable

At first , we tried filter indutries with more than 0 company from 2016 to 2018, however, the result showed kinds of NAN for R Squared,  then we tried filter more, finally find have to set filter more than 6 to make sure get R Squared value.

Convert results into the dataframe and adjusted the dataframe format to get a clean dataset about regression results.

In [276]:
years=[2016,2017,2018]
indf=calculateIndustriesRegressions(2019, years, df_c)
arr1=np.array(indf['Coefficient'].to_list())[:,0][:,0]
arr2=np.array(indf['Coefficient'].to_list())[:,0][:,1]
arr3=np.array(indf['Coefficient'].to_list())[:,0][:,2]
indf['Coefficient2016'] = arr1.tolist()
indf['Coefficient2017'] = arr2.tolist()
indf['Coefficient2018'] = arr3.tolist()
indf['Industries']=indf.iloc[:,0]
first_column = indf.pop('Industries')
indf.insert(0, 'Industries', first_column)
indf['Intercept']=np.array(indf['Intercept'].to_list())[:,0].tolist()
indf=indf.drop(indf.columns[1],axis=1)#remove repeated industries column
indf=indf.drop(columns=['Coefficient'])#remove coefficient column
indf

,Industries,OutcomeYear,MSE_test,RMSE_test,Intercept,R2_score,Coefficient2016,Coefficient2017,Coefficient2018
0,Activities auxiliary to financial intermediati...,2019,0.001666,0.040811,0.036721,-368.570788,17.976546,-36.236407,17.399672
1,Activities of membership organisation n.e.c. (91),2019,0.000210,0.014482,0.028009,0.981829,-0.133093,0.940968,0.532472
2,Air transport (62),2019,0.079894,0.282656,0.000122,0.243137,0.367418,-0.563799,1.192029
3,Chemicals nec,2019,0.005145,0.071727,0.005150,0.793452,0.372398,0.437839,0.173081
4,Computer and related activities (72),2019,0.000003,0.001760,-0.000111,0.807159,0.903596,-0.338364,0.298088
5,Construction (45),2019,0.000307,0.017532,-0.000133,0.937546,0.363022,-0.879220,1.339938
6,Extraction of crude petroleum and services rel...,2019,0.011216,0.105908,-0.018099,0.061144,0.020640,0.179937,0.608204
7,"Financial intermediation, except insurance and...",2019,0.000043,0.006520,-0.000111,0.974641,-0.307046,0.922647,0.273847
8,Manufacture of beverages,2019,0.017007,0.130413,0.008430,0.930428,-0.117300,0.298025,1.101787
9,Manufacture of electrical machinery and appara...,2019,0.000402,0.020052,-0.000177,0.891901,0.320314,-0.212506,0.837339


Check with one industry's regression, 'Activities auxiliary to financial intermediation (67)', to have a look of the regression accuracy 

In [277]:
years = [2016, 2017, 2018,2019]
df_c = df_industry_count4.copy()
for year in years:
  data = df_c[df_c['Year'] == year]
  data = data.loc[:,['CompanyName','Env_intensity','Industry(Exiobase)']]
  data.rename(columns={'Env_intensity': f'Env_intensity_{year}'}, inplace=True) 
  if(year == min(years)):
    data1 = pd.DataFrame(data)
  else:
    data2 = pd.merge(data1, data, on=["CompanyName",'Industry(Exiobase)'])
    data1 = data2.copy()
    
data = data2[data2['Industry(Exiobase)'] == 'Activities auxiliary to financial intermediation (67)']
X=data[['Env_intensity_2016','Env_intensity_2017','Env_intensity_2018']]
y=data['Env_intensity_2019']
regr = linear_model.LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
regr.fit(X_train, y_train)
y_train_pred = regr.predict(X_train)
y_pred = regr.predict(X_test)
print('MSE train: %.3f, test: %.3f' % (metrics.mean_squared_error(y_train, y_train_pred),
                metrics.mean_squared_error(y_test, y_pred)))
print('R2 score:', metrics.r2_score(y_test, y_pred))
print('Model intercept: ', regr.intercept_)
print('Model coefficients: ', regr.coef_)


MSE train: 0.002, test: 0.002
R2 score: -368.570788210546
Model intercept:  0.0367213032370731
Model coefficients:  [ 17.97654644 -36.23640739  17.3996719 ]


In [ ]:
years = [2016, 2017, 2018,2019]
df_c = df_industry_count4.copy()
for year in years:
  data = df_c[df_c['Year'] == year]
  data = data.loc[:,['CompanyName','Env_intensity','Industry(Exiobase)']]
  data.rename(columns={'Env_intensity': f'Env_intensity_{year}'}, inplace=True) 
  if(year == min(years)):
    data1 = pd.DataFrame(data)
  else:
    data2 = pd.merge(data1, data, on=["CompanyName",'Industry(Exiobase)'])
    data1 = data2.copy()
    
data = data2[data2['Industry(Exiobase)'] == 'Activities auxiliary to financial intermediation (67)']
X=data[['Env_intensity_2016','Env_intensity_2017','Env_intensity_2018']]
y=data['Env_intensity_2019']
regr = linear_model.LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
regr.fit(X_train, y_train)
y_train_pred = regr.predict(X_train)
y_pred = regr.predict(X_test)
print('MSE train: %.3f, test: %.3f' % (metrics.mean_squared_error(y_train, y_train_pred),
                metrics.mean_squared_error(y_test, y_pred)))
print('R2 score:', metrics.r2_score(y_test, y_pred))
print('Model intercept: ', regr.intercept_)
print('Model coefficients: ', regr.coef_)


The result is consistent with function result.
Then, we have a look at the result table.

In [278]:
indf=indf.set_index('Industries')
maxValuesObj = indf['R2_score'].max()
print('Maximum value in each column : ')
print(maxValuesObj)
maxValueIndexObj = indf['R2_score'].idxmax()
print("Max values of columns are at row index position :")
print(maxValueIndexObj)
minValuesObj = indf['R2_score'].min()
print('Minimum value in each column : ')
print(minValuesObj)
minValueIndexObj = indf['R2_score'].idxmin()
print("Min values of columns are at row index position :")
print(minValueIndexObj)

Maximum value in each column : 
0.9818286119890922
Max values of columns are at row index position :
Activities of membership organisation n.e.c. (91)
Minimum value in each column : 
-368.570788210546
Min values of columns are at row index position :
Activities auxiliary to financial intermediation (67)


For Activities of membership organisation n.e.c. (91), we have the highest R2_score(0.98), which indicates about 98% variance in the Environmental intensity for 2019 that the independent variables explain collectively.


If the fit is actually worse than just fitting a horizontal line then R-square is negative. Accordingly, the regression does not work for Activities auxiliary to financial intermediation (67).


In [279]:
# five largest values of indutries for R square
indf5=indf.nlargest(5, ['R2_score'])
indf5

,OutcomeYear,MSE_test,RMSE_test,Intercept,R2_score,Coefficient2016,Coefficient2017,Coefficient2018
Industries,,,,,,,,
Activities of membership organisation n.e.c. (91),2019,0.000210,0.014482,0.028009,0.981829,-0.133093,0.940968,0.532472
Research and development (73),2019,0.000024,0.004931,-0.003852,0.975646,0.805671,-0.298986,0.064527
"Financial intermediation, except insurance and pension funding (65)",2019,0.000043,0.006520,-0.000111,0.974641,-0.307046,0.922647,0.273847
"Manufacture of fabricated metal products, except machinery and equipment (28)",2019,0.000020,0.004507,0.001019,0.969378,-0.207075,0.769293,0.456634
Manufacture of machinery and equipment n.e.c. (29),2019,0.000075,0.008643,0.002781,0.958841,0.382328,-0.201888,0.978355


The dataframe shows industries with the first five largest R squared, including Activities of membership organisation n.e.c. (91), Research and development (73), Financial intermediation, except insurance and pension funding (65), Manufacture of fabricated metal products, except machinery and equipment (28), and Manufacture of machinery and equipment n.e.c. (29). 

Due to limited industries left when using years from 2016 to 2018 to forecast 2019 environmental intensity, we decide to have a look at years from 2017 and 2018 to forecast 2019 environmental intensity.

## Input data from 2017 and 2018 for independent variable

In [280]:
years=[2017,2018]
years.sort()
indf2=calculateIndustriesRegressions(2019, years, df_c)
arr1=np.array(indf2['Coefficient'].to_list())[:,0][:,0]
arr2=np.array(indf2['Coefficient'].to_list())[:,0][:,1]
indf2['Coefficient2017'] = arr1.tolist()
indf2['Coefficient2018'] = arr2.tolist()
indf2['Industries']=indf2.iloc[:,0]
first_column = indf2.pop('Industries')
indf2.insert(0, 'Industries', first_column)
indf2['Intercept']=np.array(indf2['Intercept'].to_list())[:,0].tolist()
indf2=indf2.drop(indf2.columns[1],axis=1)#remove repeated industries column
indf2=indf2.drop(columns=['Coefficient'])#remove coefficient column
indf2


,Industries,OutcomeYear,MSE_test,RMSE_test,Intercept,R2_score,Coefficient2017,Coefficient2018
0,Activities auxiliary to financial intermediati...,2019,2.433808e-04,0.015601,0.022774,-53.299135,4.261583,-2.454473
1,Activities of membership organisation n.e.c. (91),2019,1.743898e-04,0.013206,0.028261,0.979978,0.865499,0.414375
2,Air transport (62),2019,5.252998e-02,0.229194,-0.002812,0.055293,-0.416125,1.417321
3,Chemicals nec,2019,2.084507e-04,0.014438,0.001932,0.949105,-0.103712,1.103262
4,Computer and related activities (72),2019,8.755725e-04,0.029590,0.000253,-39.568634,0.503347,0.462031
5,Construction (45),2019,3.239117e-04,0.017998,0.000525,0.932321,-0.728185,1.704921
6,Extraction of crude petroleum and services rel...,2019,3.436544e-03,0.058622,-0.035540,0.793735,0.123274,0.648872
7,Extraction of natural gas and services related...,2019,4.816529e-03,0.069401,-0.009544,-1.900541,1.390601,-0.661424
8,"Financial intermediation, except insurance and...",2019,2.217217e-05,0.004709,-0.001817,0.973692,0.332456,0.500322
9,"Insurance and pension funding, except compulso...",2019,1.900960e-07,0.000436,-0.000042,NaN,0.880000,-0.020000


In [281]:
# five largest values of indutries for R square
indf2.nlargest(5, ['R2_score'])

,Industries,OutcomeYear,MSE_test,RMSE_test,Intercept,R2_score,Coefficient2017,Coefficient2018
12,"Manufacture of fabricated metal products, exce...",2019,0.000007,0.002585,0.000973,0.991924,0.400094,0.604175
17,"Manufacture of radio, television and communica...",2019,0.000007,0.002671,0.000078,0.985330,-0.182957,1.172883
1,Activities of membership organisation n.e.c. (91),2019,0.000174,0.013206,0.028261,0.979978,0.865499,0.414375
10,Manufacture of beverages,2019,0.005129,0.071619,0.009014,0.979018,0.068502,1.207932
30,Real estate activities (70),2019,0.000050,0.007097,-0.001203,0.976835,-0.529953,1.350316


In [282]:
indf25=indf2.nlargest(5, ['R2_score'])
indf25['Industries'].tolist()

['Manufacture of fabricated metal products, except machinery and equipment (28)',
 'Manufacture of radio, television and communication equipment and apparatus (32)',
 'Activities of membership organisation n.e.c. (91)',
 'Manufacture of beverages',
 'Real estate activities (70)']

The situation is a little bit different with the regression for 2016 to 2018 to researsh 2019 env intensity. However, regressions for Activities of membership organisation n.e.c. (91), and Manufacture of fabricated metal products, except machinery and equipment (28) both perform good.

In [283]:
print(indf5['R2_score'].mean())
print(indf25['R2_score'].mean())

0.9720669285619052
0.9826170978729676


In [284]:
print(indf5['MSE_test'].mean())
print(indf25['MSE_test'].mean())

7.431229530659072e-05
0.0010735819826167424


With the higher R2 score and lower MSE for test data, regressions with 2017 and 2018 data is better than with data 2016 to 2018 for the first five industries.

# Industry regressions--Evironmental Intensity with growth rate

Now, we did the regression for environmental intensity with growth rate to check whether the regression performs better

In [285]:
df_c = df_industry_count4.copy()
def calculateIndustriesGrowthRegressions(outcomeYear, pastYears, df_c):
    industry_regressions = {}
    for i in np.unique(df_c['Industry(Exiobase)']):
        for year in years:
            years.sort()
            data = df_c[(df_c['Industry(Exiobase)'] == i) & (df_c['Year'] == year)]
            data = data.loc[:,['CompanyName','Env_intensity','Environmental_Growth']]
            data.rename(columns={'Env_intensity': f'Env_intensity_{year}','Environmental_Growth': f'Environmental_Growth_{year}'}, inplace=True) 
            if(year == min(years)):
                data1 = pd.DataFrame(data)
            else:
                data2 = pd.merge(data1, data, on=["CompanyName"])
                data1 = data2.copy()
        data2.dropna(inplace=True)
        data2 = data2.fillna("", inplace=False)
        if (data2.shape[0])> 6:
            data3 = df_c[df_c['Year'] == outcomeYear]
            data3 = data3[['CompanyName','Env_intensity']]
            data3.rename(columns={'Env_intensity': f'Env_intensity_{outcomeYear}'}, inplace=True) 
            data3 = pd.merge(data3, data2, on=["CompanyName"])

            filter_col = [col for col in data3 if ((col.startswith('Env_intensity') and not(col.endswith(f'{outcomeYear}'))))or ((col.startswith('Environmental_Growth_') and not(col.endswith(f'{outcomeYear}'))))]
            outcome_col = [col for col in data3 if (col.startswith('Env_intensity') and col.endswith(f'{outcomeYear}'))]
            X=data3[filter_col]
            y=data3[outcome_col]

            regr = linear_model.LinearRegression()
            X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
            regr.fit(X_train, y_train)
            y_train_pred = regr.predict(X_train)
            y_pred = regr.predict(X_test)
        else: 
            continue
        industry_regressions[i] = {'OutcomeYear':outcomeYear, 'MSE_test': metrics.mean_squared_error(y_test, y_pred),'RMSE_test':sqrt(metrics.mean_squared_error(y_test, y_pred)),'Intercept': regr.intercept_,'Coefficient': regr.coef_, 
                                   'R2_score':metrics.r2_score(y_test, y_pred)}

    data_items = industry_regressions.items()
    data_list = list(data_items)

    df = pd.DataFrame(data_list)

    df=pd.concat([df.drop(columns=df.columns[1], axis=1), df.iloc[:,1].apply(pd.Series)], axis=1)

    return df

## Input data from 2016 to 2018

In [286]:
years = [2016, 2017, 2018]
gindf=calculateIndustriesGrowthRegressions(2019, years, df_c)
arr1=np.array(gindf['Coefficient'].to_list())[:,0][:,0]
arr2=np.array(gindf['Coefficient'].to_list())[:,0][:,1]
arr3=np.array(gindf['Coefficient'].to_list())[:,0][:,2]
arr4=np.array(gindf['Coefficient'].to_list())[:,0][:,3]
arr5=np.array(gindf['Coefficient'].to_list())[:,0][:,4]
arr6=np.array(gindf['Coefficient'].to_list())[:,0][:,5]
gindf['Coefficient2016'] = arr1.tolist()
gindf['Coefficient2017'] = arr2.tolist()
gindf['Coefficient2018'] = arr3.tolist()
gindf['Coefficientgrowth2016'] = arr4.tolist()
gindf['Coefficientgrowth2017'] = arr5.tolist()
gindf['Coefficientgrowth2018'] = arr6.tolist()
gindf['Industries']=gindf.iloc[:,0]
first_column = gindf.pop('Industries')
gindf.insert(0, 'Industries', first_column)
gindf['Intercept']=np.array(gindf['Intercept'].to_list())[:,0].tolist()
gindf=gindf.drop(gindf.columns[1],axis=1)#remove repeated industries column
gindf=gindf.drop(columns=['Coefficient'])#remove coefficient column
gindf

,Industries,OutcomeYear,MSE_test,RMSE_test,Intercept,R2_score,Coefficient2016,Coefficient2017,Coefficient2018,Coefficientgrowth2016,Coefficientgrowth2017,Coefficientgrowth2018
0,Activities auxiliary to financial intermediati...,2019,8.145607e-03,0.090253,0.184351,-1806.457281,-76.671009,5.231518e-03,299.823319,1.188214e-02,-208.056443,-0.009757
1,Activities of membership organisation n.e.c. (91),2019,1.817273e-04,0.013481,-0.014931,0.984255,-1.441925,-5.700398e-04,2.637142,3.600161e-05,-0.193111,-0.001095
2,Air transport (62),2019,1.565534e-01,0.395668,0.004236,-0.483075,3.718195,-8.604827e-03,-4.327460,-7.325014e-03,1.546423,-0.000039
3,Chemicals nec,2019,5.203184e-04,0.022810,-0.002510,0.743259,-0.791492,-7.529472e-05,0.360464,1.375325e-03,1.417159,0.000169
4,Computer and related activities (72),2019,3.707767e-06,0.001926,-0.000160,0.715955,0.292750,-5.419177e-06,0.824918,3.782132e-05,-0.269905,-0.000036
5,Construction (45),2019,3.258664e-05,0.005708,0.002028,0.967457,-0.236089,-7.235401e-06,-0.335368,1.002582e-04,1.524905,0.000108
6,Extraction of crude petroleum and services rel...,2019,3.995132e-03,0.063207,-0.088699,0.481081,0.718391,-5.110948e-04,-0.124293,-2.503166e-03,-0.038587,-0.002001
7,"Financial intermediation, except insurance and...",2019,2.019961e-05,0.004494,0.000522,0.596149,-0.480050,2.765622e-05,0.499465,2.569836e-05,0.862389,0.000220
8,Manufacture of beverages,2019,1.004193e-02,0.100209,0.000235,0.958872,0.092050,2.111523e-04,-0.651669,-4.401866e-05,1.602156,0.000207
9,Manufacture of electrical machinery and appara...,2019,4.520346e-04,0.021261,-0.004139,0.878469,0.529052,-9.172356e-05,-0.204613,-3.618167e-04,0.586138,-0.000247


check with one industry whether the function is accurate

In [287]:
years = [2016, 2017, 2018,2019]
df_c = df_industry_count4.copy()
for year in years:
  data = df_c[df_c['Year'] == year]
  data = data.loc[:,['CompanyName','Env_intensity','Industry(Exiobase)','Environmental_Growth']]
  data.rename(columns={'Env_intensity': f'Env_intensity_{year}','Environmental_Growth': f'Environmental_Growth_{year}'}, inplace=True) 
  if(year == min(years)):
    data1 = pd.DataFrame(data)
  else:
    data2 = pd.merge(data1, data, on=["CompanyName",'Industry(Exiobase)'])
    data1 = data2.copy()
data = data2[data2['Industry(Exiobase)'] == 'Air transport (62)']
X=data.drop(columns=['CompanyName','Env_intensity_2019','Industry(Exiobase)','Environmental_Growth_2019'])
X=data[['Env_intensity_2016','Environmental_Growth_2016','Env_intensity_2017','Environmental_Growth_2017','Env_intensity_2018','Environmental_Growth_2018']]
y=data['Env_intensity_2019']
regr = linear_model.LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
regr.fit(X_train, y_train)
y_train_pred = regr.predict(X_train)
y_pred = regr.predict(X_test)
print('MSE train: %.3f, test: %.3f' % (metrics.mean_squared_error(y_train, y_train_pred),
                metrics.mean_squared_error(y_test, y_pred)))
print('R2 score:', metrics.r2_score(y_test, y_pred))
print('Model intercept: ', regr.intercept_)
print('Model coefficients: ', regr.coef_)


MSE train: 0.000, test: 0.157
R2 score: -0.4830752229570636
Model intercept:  0.004235999337293073
Model coefficients:  [ 3.71819540e+00 -8.60482710e-03 -4.32745978e+00 -7.32501446e-03
  1.54642276e+00 -3.88414716e-05]


The result is consistent with the table

In [288]:
# five largest values of indutries for R square
gindf5=gindf.nlargest(5, ['R2_score'])
gindf5

,Industries,OutcomeYear,MSE_test,RMSE_test,Intercept,R2_score,Coefficient2016,Coefficient2017,Coefficient2018,Coefficientgrowth2016,Coefficientgrowth2017,Coefficientgrowth2018
16,Manufacture of rubber and plastic products (25),2019,5.478148e-07,0.000740,0.006995,0.995124,-0.163042,0.000373,4.533070,-0.000099,-3.151470,-0.000760
28,"Recreational, cultural and sporting activities...",2019,3.447179e-06,0.001857,0.000945,0.985070,0.239191,0.000060,-0.377784,0.000014,1.078239,0.000012
1,Activities of membership organisation n.e.c. (91),2019,1.817273e-04,0.013481,-0.014931,0.984255,-1.441925,-0.000570,2.637142,0.000036,-0.193111,-0.001095
5,Construction (45),2019,3.258664e-05,0.005708,0.002028,0.967457,-0.236089,-0.000007,-0.335368,0.000100,1.524905,0.000108
10,"Manufacture of fabricated metal products, exce...",2019,2.605114e-05,0.005104,-0.000575,0.961553,-0.256612,0.000099,1.883722,0.000161,-0.649369,-0.000307


In [289]:
gindf5['Industries'].tolist()

['Manufacture of rubber and plastic products (25)',
 'Recreational, cultural and sporting activities (92)',
 'Activities of membership organisation n.e.c. (91)',
 'Construction (45)',
 'Manufacture of fabricated metal products, except machinery and equipment (28)']

In [290]:
print(gindf5['R2_score'].mean())
print(indf5['R2_score'].mean())

0.978691993942775
0.9720669285619052


In [291]:
print(gindf5['MSE_test'].mean())
print(indf5['MSE_test'].mean())

4.887200983412902e-05
7.431229530659072e-05


When comparing with the first 5 industries ordered by R2 score, it seems the regression with growth rate performs better because of the higher R squared and lower MSE to test data.

Then we are going to compare with the regression by using data from 2017 and 2018 to forecast

## Input data from 2017 and 2018

In [299]:
years = [2017, 2018]
gindf2=calculateIndustriesGrowthRegressions(2019, years, df_c)
arr1=np.array(gindf2['Coefficient'].to_list())[:,0][:,0]
arr2=np.array(gindf2['Coefficient'].to_list())[:,0][:,1]
arr3=np.array(gindf2['Coefficient'].to_list())[:,0][:,2]
arr4=np.array(gindf2['Coefficient'].to_list())[:,0][:,3]
gindf2['Coefficient2017'] = arr1.tolist()
gindf2['Coefficient2018'] = arr2.tolist()
gindf2['Coefficientgrowth2017'] = arr3.tolist()
gindf2['Coefficientgrowth2018'] = arr4.tolist()
gindf2['Industries']=gindf2.iloc[:,0]
first_column = gindf2.pop('Industries')
gindf2.insert(0, 'Industries', first_column)
gindf2['Intercept']=np.array(gindf2['Intercept'].to_list())[:,0].tolist()
gindf2=gindf2.drop(gindf2.columns[1],axis=1)#remove repeated industries column
gindf2=gindf2.drop(columns=['Coefficient'])#remove coefficient column
gindf2


,Industries,OutcomeYear,MSE_test,RMSE_test,Intercept,R2_score,Coefficient2017,Coefficient2018,Coefficientgrowth2017,Coefficientgrowth2018
0,Activities auxiliary to financial intermediati...,2019,3.648323e-03,0.060401,0.073740,-808.539089,-1.821584e+00,0.004127,2.688033e+00,-2.260783e-03
1,Activities of membership organisation n.e.c. (91),2019,2.300809e-02,0.151684,0.003164,-1.641548,1.866384e+00,0.000079,-7.247527e-01,-2.402852e-03
2,Air transport (62),2019,5.262270e-02,0.229396,-0.010415,-0.217167,1.338163e-01,-0.000368,8.413491e-01,-1.390586e-03
3,Chemicals nec,2019,3.869982e-04,0.019672,0.007529,0.830493,1.104673e-02,0.000518,9.870103e-01,-1.476433e-05
4,Computer and related activities (72),2019,1.924340e-06,0.001387,-0.002482,0.903975,2.894210e+00,-0.000011,-2.455119e+00,-1.511477e-04
5,Construction (45),2019,3.310563e-04,0.018195,0.000552,0.932729,-1.044670e+00,0.000058,1.895110e+00,2.202825e-04
6,Extraction of crude petroleum and services rel...,2019,7.255946e-03,0.085182,-0.085129,0.749181,3.469085e-01,-0.000096,1.852841e-01,-1.494215e-03
7,Extraction of natural gas and services related...,2019,2.052729e-02,0.143273,-0.119686,-11.361650,-3.732676e+00,-0.001301,3.645434e+00,4.838816e-03
8,"Financial intermediation, except insurance and...",2019,1.593699e-04,0.012624,0.000447,0.984304,2.692066e-01,-0.000017,6.235444e-01,1.470719e-04
9,"Insurance and pension funding, except compulso...",2019,2.533731e-05,0.005034,-0.003016,NaN,5.604575e-09,-0.000068,5.166575e-09,-1.472637e-05


In [312]:
# five largest values of indutries for R square
gindf25=gindf2.nlargest(5, ['R2_score'])
gindf25

,Industries,OutcomeYear,MSE_test,RMSE_test,Intercept,R2_score,Coefficient2017,Coefficient2018,Coefficientgrowth2017,Coefficientgrowth2018
10,Manufacture of beverages,2019,4.419836e-04,0.021023,0.007999,0.998192,0.015528,0.000010,1.241038,3.335071e-05
32,Renting of machinery and equipment without ope...,2019,3.115047e-07,0.000558,-0.000465,0.991790,-1.200695,-0.000010,2.116699,1.007476e-04
13,Manufacture of machinery and equipment n.e.c. ...,2019,1.706126e-05,0.004131,0.003181,0.990599,0.594594,0.000056,0.561005,-4.094574e-05
17,"Manufacture of radio, television and communica...",2019,3.195515e-06,0.001788,-0.000127,0.989351,-0.004038,-0.000022,0.981032,8.940465e-07
23,Other service activities (93),2019,1.165047e-04,0.010794,-0.006368,0.986606,-0.089004,0.000200,0.956525,1.990270e-04


In [313]:
gindf25['Industries'].tolist()

['Manufacture of beverages',
 'Renting of machinery and equipment without operator and of personal and household goods (71)',
 'Manufacture of machinery and equipment n.e.c. (29)',
 'Manufacture of radio, television and communication equipment and apparatus (32)',
 'Other service activities (93)']

In [302]:
print(gindf25['R2_score'].mean())
print(gindf5['R2_score'].mean())

0.9913073627104925
0.978691993942775


In [303]:
print(gindf25['MSE_test'].mean())
print(gindf5['MSE_test'].mean())

0.00011581132909609158
4.887200983412902e-05


Accordingly, for the first five industries, the regression with data in 2017 and 2018 performs better than the regression with data from 2016 to 2018.

In [304]:
print(gindf25['R2_score'].mean())
print(indf25['R2_score'].mean())

0.9913073627104925
0.9826170978729676


In [305]:
print(gindf25['MSE_test'].mean())
print(indf25['MSE_test'].mean())

0.00011581132909609158
0.0010735819826167424


For first five indutries ordered by R2 score, it seems be the same situation that regressions perfrom better when adding growth rates with the higher R2 score and MSE to test data.

# Conclusion

* Among first five industries ordered by R2 scores, the average result from regressions for industires with data from 2017 and 2018 perform better than regressions with data from 2016 to 2018 to forecast 2019.

* When adding growth rates as variables, the performance of regressions will be better.

* Time series model for the indusry "Manufacture of fabricated metal products, except machinery and equipment (28)" performs better than other industries that all regressions have a high R square.